In [ ]:
"""
Equipo 3
"""

from mpi4py import MPI
import RPi.GPIO as GPIO
import time as time
import picamera
import os
import subprocess
import EjecutarCamara
# Set up GPIO
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
TRIG=23
ECHO=24
#PINBUZZER=18
inicia_pulso=0
termina_pulso=0

GPIO.setup(PINBUZZER, GPIO.OUT)
GPIO.setup(TRIG, GPIO.OUT)
GPIO.setup(ECHO, GPIO.IN)
GPIO.output(TRIG, False) #Setear transmisor en false



# MPI variables for communication, rank, size, and name of the node
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name()
print "Esperando lectura  del sensor"
time.sleep(1)


print "rank=",rank
# This is the master node.
if rank == 0:
    process = 1
    while True:

        #Trigger inicia el pulso
        GPIO.output(TRIG, True)
        #Duracion del pulso 10 micro segundos
        time.sleep(0.00001)
        #Desactiva el pulso
        GPIO.output(TRIG, False)
        #Se guarda el tiempo en segundos
        inicia_pulso = time.time()

        #Muentras la recepcion (echo) este en 0,toma el tiempo del inicio del pulso
        while GPIO.input(ECHO)==0:
            inicia_pulso = time.time()
        #Cuando se reciba el rebote del pulso en echo, se toma el tiempo de final del pulso
        while GPIO.input(ECHO)==1:
            termina_pulso = time.time()
        #Calcular el tiempo que transcurrio
           duracion  = termina_pulso - inicia_pulso
        #Calculo de distancia (tiempo * velocidad del sonido cm/s)/2  **porque es ida y vuelta
        distancia = duracion*34300
        distancia= distancia/2
        #Condicion para envio de mensaje
        if distancia<=200:

            print "Distancia menor es:",distancia,"cm"
            # Envía la distancia  a todos los procesos
            for p in range(1,size):
                comm.send(distancia, dest=p, tag=1)
                print "Enviando distancia",distancia," a proceso ",process
            # recibe la información de cada nodo
            for p in range(1,size):
                print ("se recibe respuesta"+str (comm.recv(source=p,tag=1))+" de proceso"+str(p))
                # a implementar aquí lo que se hará con la información proveniente del proceso p
        else:
            # sino envía una señal de parada 
            comm.send(-1, dest=p, tag=1)




        time.sleep(0.5)
    GPIO.cleanup()


#Si no es el maestro  ejecutar la cámara en el futuro solo seria asignarle un nombre genérico a este script 
# y en cada raspi se ejecutará un script diferentes 
else:
    EjecutarCamara.Ejecutar_camara(comm)


GPIO.cleanup()


# Ejecutar cámara ejemplode script

In [ ]:
import picamera
from mpi4py import MPI

def Ejecutar_camara(comm):
    rank = comm.Get_rank()
    size = comm.Get_size()
    while True:
        print("bucleeee")
        request = comm.recv(source=0, tag=1)
        if request==-1:
            #comm.send(str(rank)+" Termina", dest=0, tag=1)
            break

        comm.send([0.0,0,0,0,0], dest=0, tag=1)
        camara = picamera.PiCamera()
        camara.capture("test"+str(request)+".jpg")
        print ("se recibe peticion"+str (request))
        camara.close()